In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
data=pd.read_csv("Bank_Personal_Loan_Modelling.csv")
y=data["Personal Loan"]
data["Loan"]=y
data.drop(columns="Personal Loan",inplace=True)
X = data.iloc[:,:-1]
y = data.iloc[:,-1]
enc = OneHotEncoder()
Y = enc.fit_transform(y[:, np.newaxis]).toarray()



C:\Users\monik\AppData\Local\Temp\ipykernel_36232\3425334974.py:8: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  Y = enc.fit_transform(y[:, np.newaxis]).toarray()


In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.5, random_state=2)

In [5]:
def create_model():
    model = Sequential()
    model.add(Dense(units=10, input_dim=13, activation='relu'))
    model.add(Dense(units=10, activation='relu'))
    model.add(Dense(units=2, activation='softmax'))
    optimizer = Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
    

In [6]:
def objective_function(model):
    loss, accuracy = model.evaluate(X_test,Y_test,verbose=0)
    return accuracy


In [18]:
population_size = 5
mutation_rate = 0.1
crossover_rate = 0.9
generations = 5
population = []
for i in range(population_size):
    model = create_model()
    weights = model.get_weights()
    population.append(weights)
    
def evaluate_fitness(x):
    model.set_weights(x)
    loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    return accuracy

for i in range(generations):
  
    fitness_values = []
    for j in range(population_size):
        model = create_model()
        model.set_weights(population[j])
        fitness_values.append(objective_function(model))
    belief_space=[]

    fittest_individuals = np.argsort(fitness_values)[-population_size:]
    
    
    
    # update belief space
    best_index = np.argmax(fitness_values)
    best_solution = population[best_index]
    belief_space += [best_solution]
    belief_space = belief_space[:population_size]
    

    offspring = []
    for j in range(population_size):
        parent1_index = np.random.choice(fittest_individuals)
        parent2_index = np.random.choice(fittest_individuals)
        parent1 = population[parent1_index]
        parent2 = population[parent2_index]
        offspring_weights = []
        for k in range(len(parent1)):
            if np.random.uniform() < crossover_rate:
                offspring_weights.append(parent1[k])
            else:
                offspring_weights.append(parent2[k])
        for k in range(len(offspring_weights)):
            if np.random.uniform() < mutation_rate:
                shape = offspring_weights[k].shape
                offspring_weights[k] = offspring_weights[k] + np.random.normal(size=shape)
        offspring.append(offspring_weights)

  
    offspring_fitness_values = []
    for j in range(len(offspring)):
        model = create_model()
        model.set_weights(offspring[j])
        offspring_fitness_values.append(objective_function(model))

    combined_population = population + offspring
    combined_fitness_values = fitness_values + offspring_fitness_values


    population_indices = np.argsort(combined_fitness_values)[-population_size:]
    population = [combined_population[i] for i in population_indices]
    
    #set the best solution to belief space
    if evaluate_fitness(belief_space[0]) > evaluate_fitness(best_solution):
        best_solution = belief_space[0]
    else:
        belief_space[-1] = best_solution
    
    combined_population+=belief_space[0]

    print("Generation ", i+1, " Best Fitness Value: ", np.max(combined_fitness_values))


Generation  1  Best Fitness Value:  0.8988000154495239
Generation  2  Best Fitness Value:  0.8988000154495239
Generation  3  Best Fitness Value:  0.909600019454956
Generation  4  Best Fitness Value:  0.909600019454956
Generation  5  Best Fitness Value:  0.909600019454956
